In [1]:
from imutils import paths
import cv2
import os
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

import torch.autograd as autograd

In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
import torch
device = torch.device("cuda")

In [4]:
# image_paths = list(paths.list_images('data/Caltech101/001'))
# image_paths = list(paths.list_images('data/cars_side-view'))
image_paths = list(paths.list_images('data/Caltech101/016'))

In [5]:
data = []
labels = []
for img_path in tqdm(image_paths):
    label = img_path.split(os.path.sep)[-2]
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    data.append(img)
    labels.append(label)
    if len(labels) > 5000:
        break
    
data = np.array(data)
labels = np.array(labels)

100%|██████████| 123/123 [00:00<00:00, 2668.46it/s]


In [6]:

lb = LabelEncoder()
labels = lb.fit_transform(labels)
print(f"Total Number of Classes: {len(lb.classes_)}")

Total Number of Classes: 1


In [7]:
Counter(labels)

Counter({0: 123})

In [8]:
from sklearn.model_selection import train_test_split
# divide the data into train and test set
(x_train, x_test, y_train, y_test) = train_test_split(data, labels, test_size=0.1, stratify=labels, random_state=42)
print(f"x_train examples: {x_train.shape}\nx_test examples: {x_test.shape}")

x_train examples: (110, 197, 300, 3)
x_test examples: (13, 197, 300, 3)


In [9]:
dataset_config = {'size': 64, 'channels': 3, 'classes': 1}

In [10]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((dataset_config['size'], dataset_config['size'])),
    transforms.ToTensor(),
    #transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((dataset_config['size'],dataset_config['size'])),
    transforms.ToTensor(),
    #transforms.Normalize(mean = [0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])    

In [11]:
BS = 32
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=BS, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train=False, transform=transforms.ToTensor()),
    batch_size=BS, shuffle=True)


Files already downloaded and verified


In [12]:
BS = 32
# custom dataset class
class CustomDataset(Dataset):
    def __init__(self, images, labels= None, transforms = None):
        self.labels = labels
        self.images = images
        self.transforms = transforms
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        data = self.images[index][:]
        
        if self.transforms:
            data = self.transforms(data)
            
        
        return (data, self.labels[index])
        
train_data = CustomDataset(x_train, y_train, train_transforms)
test_data = CustomDataset(x_test, y_test, val_transform)       

train_loader = DataLoader(train_data, batch_size=BS, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=BS, shuffle=True, num_workers=4, drop_last=False) 

### --- Main --

In [13]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable

from torchvision.utils import save_image

def gradients(y, x):
    return autograd.grad(
                outputs=y, inputs=x, retain_graph=True,
                create_graph=True, grad_outputs=torch.ones_like(y), only_inputs=True)[0]

In [14]:
class HWReduction(nn.Module):
    def forward(self, x):
        # x -> [B, C, H, W]
        return x.mean(dim=(-1, -2))

class Reshape(nn.Module):
    def __init__(self, shape: list):
        super(Reshape, self).__init__()
        self.shape = shape
    def forward(self, x):
        batch_size = x.shape[0]
        return x.reshape([batch_size] + self.shape)

class VAE_Cifar10(nn.Module):
    def __init__(self, label = 'cifar10', image_size = dataset_config['size'],
                 channel_num = dataset_config['channels'],
                 z_size=128):
        # configurations
        super().__init__()
        self.label = label
        self.image_size = image_size
        self.channel_num = channel_num
        self.z_size = z_size

        # encoder
        self.encoder = nn.Sequential(
            self.capacity_conv(channel_num, 16), # 16 x 64 x 64
            nn.InstanceNorm2d(16),
            self.downsampling_conv(16, 32), # 32 x 32 x 32
            self.capacity_conv(32, 64), # 64 x 32 x 32
            self.downsampling_conv(64, 128), # 128 x 16 x 16
            self.capacity_conv(128, 256), # 256 x 16 x 16
            self.downsampling_conv(256, 512), # 512 x 8 x 8
            HWReduction(),
        )

        # H, W will be reduced


        # q
        self.q_mean = self._linear(512, z_size, relu=False)
        self.q_logvar = self._linear(512, z_size, relu=False)

        # projection
        self.project = nn.Sequential(
            self._linear(z_size, 1024),
            self._linear(1024, 8 * 8 * 128),
            Reshape([128, 8, 8])
        )

        # decoder
        self.decoder = nn.Sequential(
            self.upsampling_conv(128, 64), # 32 x 16 x 16
            self.capacity_conv(64, 64), # 64 x 16 x 16
            self.upsampling_conv(64, 32), # 32 x 32 x 32
            self.capacity_conv(32, 32), # 32 x 32 x 32
            self.upsampling_conv(32, 16), # 16 x 64 x 64
            nn.Conv2d(
                16, channel_num,
                kernel_size=3, stride=1, padding=1,
            ),
            nn.Sigmoid()
        )


    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std


    def forward(self, x):
        encoded = self.encoder(x)

        # sample latent code z from q given x.
        mean, logvar = self.q_mean(encoded), self.q_logvar(encoded) - 10.0
        z = self.reparameterize(mean, logvar)
        z_projected = self.project(z)

        # reconstruct x from z
        x_reconstructed = self.decoder(z_projected)
        return x_reconstructed, mean, logvar
    
    # ======
    # Layers
    # ======

    def downsampling_conv(self, channel_size, kernel_num):
        return nn.Sequential(
            nn.Conv2d(
                channel_size, kernel_num,
                kernel_size=4, stride=2, padding=1,
            ),
            nn.InstanceNorm2d(kernel_num),
            nn.LeakyReLU(negative_slope=0.1),
        )
    def capacity_conv(self, channel_num, kernel_num):
        return nn.Sequential(
            nn.Conv2d(
                channel_num, kernel_num,
                kernel_size=3, stride=1, padding=1,
            ),
            nn.LeakyReLU(negative_slope=0.1),
        )

    def upsampling_conv(self, channel_num, kernel_num):
        return nn.Sequential(
            nn.ConvTranspose2d(
                channel_num, kernel_num,
                kernel_size=4, stride=2, padding=1,
            ),
            nn.InstanceNorm2d(kernel_num),
            nn.LeakyReLU(negative_slope=0.1),
        )

    def _linear(self, in_size, out_size, relu=True):
        return nn.Sequential(
            nn.Linear(in_size, out_size),
            nn.ReLU(),
        ) if relu else nn.Linear(in_size, out_size)

model = VAE_Cifar10().to(device)
optimizer = optim.AdamW(model.parameters(), lr=3e-4)

In [15]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    MSE = (recon_x - x.view(-1,  dataset_config['channels'],dataset_config['size'],dataset_config['size'])) ** 2
    MSE = MSE.sum(dim=(-1,-2,-3))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=-1)

    return MSE, KLD

In [16]:
def train(epoch, log_eta):
    model.train()
    log_eta, just_updated = log_eta
    train_recons_loss = 0
    train_kld_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        recons_loss, kld_loss = loss_function(recon_batch, data, mu, logvar)
        loss = (recons_loss + float(np.exp(log_eta)) * kld_loss).mean(dim=0)
        loss.backward()
        train_recons_loss += recons_loss.mean(dim=0).item()
        train_kld_loss += kld_loss.mean(dim=0).item()
        optimizer.step()
        if batch_idx % 2 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tRecons Loss: {:.6f}; KLD Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                recons_loss.mean(dim=0).item(), recons_loss.mean(dim=0).item()))
    
    print('====> Epoch: {} Average recons loss: {:.4f}, kld loss: {:.4f}'.format(
          epoch, train_recons_loss / len(train_loader.dataset), train_kld_loss / len(train_loader.dataset)))
    if train_recons_loss / len(train_loader.dataset) < 6.0 and (epoch - just_updated) >= 10 and (train_recons_loss / len(train_loader.dataset)) < train_kld_loss / len(train_loader.dataset):
        log_eta += 0.25
        if log_eta >= 0.0:
            log_eta = 0.0
        just_updated = epoch
        print('====> Epoch: {} Eta is now {:.4f} due to sufficient recons results'.format(epoch, float(np.exp(log_eta))))
    return (log_eta, just_updated)

In [17]:
def test(epoch):
    model.eval()
    test_recons_loss = 0
    test_kld_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            recons_loss, kld_loss = loss_function(recon_batch, data, mu, logvar)
            test_recons_loss += recons_loss.mean(dim=0).item()
            test_kld_loss += kld_loss.mean(dim=0).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch[:n].view(n,  dataset_config['channels'], dataset_config['size'], dataset_config['size'])[:n]])
                if not os.path.exists("results/"):
                    os.mkdir("results")
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)
        for i, (data, _) in enumerate(train_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch[:n].view(n,  dataset_config['channels'], dataset_config['size'], dataset_config['size'])[:n]])
                if not os.path.exists("overfit_results/"):
                    os.mkdir("overfit_results")
                save_image(comparison.cpu(),
                         'overfit_results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_recons_loss /= len(test_loader.dataset)
    test_kld_loss /= len(test_loader.dataset)
    print('====> Test set recons loss: {:.4f}, kld_loss: {:.4f}'.format(test_recons_loss, test_kld_loss))

In [18]:
log_eta = (-3., 0)
for epoch in range(1, 1000 + 1):
    log_eta = train(epoch, log_eta)
    if epoch % 20 == 0:
        test(epoch)

Train Epoch: 1 [0/110 (0%)]	Recons Loss: 602.973755; KLD Loss: 602.973755
Train Epoch: 1 [64/110 (50%)]	Recons Loss: 589.637573; KLD Loss: 589.637573
====> Epoch: 1 Average recons loss: 21.3870, kld loss: 41.7312
====> Test set recons loss: 41.4171, kld_loss: 87.0031
Train Epoch: 2 [0/110 (0%)]	Recons Loss: 501.622375; KLD Loss: 501.622375
Train Epoch: 2 [64/110 (50%)]	Recons Loss: 625.666687; KLD Loss: 625.666687
====> Epoch: 2 Average recons loss: 19.1828, kld loss: 40.7559
====> Test set recons loss: 39.4873, kld_loss: 84.8462
Train Epoch: 3 [0/110 (0%)]	Recons Loss: 539.166626; KLD Loss: 539.166626
Train Epoch: 3 [64/110 (50%)]	Recons Loss: 493.024567; KLD Loss: 493.024567
====> Epoch: 3 Average recons loss: 18.7149, kld loss: 39.7263
====> Test set recons loss: 38.0889, kld_loss: 82.6257
Train Epoch: 4 [0/110 (0%)]	Recons Loss: 543.895508; KLD Loss: 543.895508
Train Epoch: 4 [64/110 (50%)]	Recons Loss: 495.436340; KLD Loss: 495.436340
====> Epoch: 4 Average recons loss: 18.0295, k

====> Test set recons loss: 27.3322, kld_loss: 62.9920
Train Epoch: 32 [0/110 (0%)]	Recons Loss: 186.011566; KLD Loss: 186.011566
Train Epoch: 32 [64/110 (50%)]	Recons Loss: 170.248108; KLD Loss: 170.248108
====> Epoch: 32 Average recons loss: 6.2441, kld loss: 29.9525
====> Test set recons loss: 29.6751, kld_loss: 63.2534
Train Epoch: 33 [0/110 (0%)]	Recons Loss: 168.512192; KLD Loss: 168.512192
Train Epoch: 33 [64/110 (50%)]	Recons Loss: 177.907471; KLD Loss: 177.907471
====> Epoch: 33 Average recons loss: 6.0061, kld loss: 30.0889
====> Test set recons loss: 28.0845, kld_loss: 63.5753
Train Epoch: 34 [0/110 (0%)]	Recons Loss: 175.476868; KLD Loss: 175.476868
Train Epoch: 34 [64/110 (50%)]	Recons Loss: 156.874329; KLD Loss: 156.874329
====> Epoch: 34 Average recons loss: 5.9269, kld loss: 30.2944
====> Epoch: 34 Eta is now 0.0639 due to sufficient recons results
====> Test set recons loss: 28.1702, kld_loss: 64.0722
Train Epoch: 35 [0/110 (0%)]	Recons Loss: 153.029358; KLD Loss: 153.

====> Test set recons loss: 30.8820, kld_loss: 55.2483
Train Epoch: 62 [0/110 (0%)]	Recons Loss: 94.405807; KLD Loss: 94.405807
Train Epoch: 62 [64/110 (50%)]	Recons Loss: 85.979446; KLD Loss: 85.979446
====> Epoch: 62 Average recons loss: 3.3892, kld loss: 26.2661
====> Test set recons loss: 30.7893, kld_loss: 54.7038
Train Epoch: 63 [0/110 (0%)]	Recons Loss: 96.738770; KLD Loss: 96.738770
Train Epoch: 63 [64/110 (50%)]	Recons Loss: 90.551598; KLD Loss: 90.551598
====> Epoch: 63 Average recons loss: 3.3724, kld loss: 26.0615
====> Test set recons loss: 31.8315, kld_loss: 54.4563
Train Epoch: 64 [0/110 (0%)]	Recons Loss: 93.128807; KLD Loss: 93.128807
Train Epoch: 64 [64/110 (50%)]	Recons Loss: 88.884651; KLD Loss: 88.884651
====> Epoch: 64 Average recons loss: 3.3228, kld loss: 25.9224
====> Epoch: 64 Eta is now 0.1353 due to sufficient recons results
====> Test set recons loss: 30.7801, kld_loss: 54.2560
Train Epoch: 65 [0/110 (0%)]	Recons Loss: 81.023628; KLD Loss: 81.023628
Train E

Train Epoch: 92 [0/110 (0%)]	Recons Loss: 76.795212; KLD Loss: 76.795212
Train Epoch: 92 [64/110 (50%)]	Recons Loss: 89.420975; KLD Loss: 89.420975
====> Epoch: 92 Average recons loss: 3.0202, kld loss: 19.2626
====> Test set recons loss: 33.4273, kld_loss: 39.8185
Train Epoch: 93 [0/110 (0%)]	Recons Loss: 81.173935; KLD Loss: 81.173935
Train Epoch: 93 [64/110 (50%)]	Recons Loss: 80.891006; KLD Loss: 80.891006
====> Epoch: 93 Average recons loss: 2.9537, kld loss: 19.2875
====> Test set recons loss: 33.0197, kld_loss: 39.4449
Train Epoch: 94 [0/110 (0%)]	Recons Loss: 79.846756; KLD Loss: 79.846756
Train Epoch: 94 [64/110 (50%)]	Recons Loss: 85.998634; KLD Loss: 85.998634
====> Epoch: 94 Average recons loss: 2.9361, kld loss: 19.1298
====> Epoch: 94 Eta is now 0.2865 due to sufficient recons results
====> Test set recons loss: 32.0820, kld_loss: 39.0583
Train Epoch: 95 [0/110 (0%)]	Recons Loss: 80.799835; KLD Loss: 80.799835
Train Epoch: 95 [64/110 (50%)]	Recons Loss: 86.487244; KLD Los

====> Test set recons loss: 35.9524, kld_loss: 25.5172
Train Epoch: 122 [0/110 (0%)]	Recons Loss: 100.518570; KLD Loss: 100.518570
Train Epoch: 122 [64/110 (50%)]	Recons Loss: 99.953278; KLD Loss: 99.953278
====> Epoch: 122 Average recons loss: 3.7119, kld loss: 12.7791
====> Test set recons loss: 38.1531, kld_loss: 25.0911
Train Epoch: 123 [0/110 (0%)]	Recons Loss: 103.563225; KLD Loss: 103.563225
Train Epoch: 123 [64/110 (50%)]	Recons Loss: 95.995728; KLD Loss: 95.995728
====> Epoch: 123 Average recons loss: 3.7070, kld loss: 12.7635
====> Test set recons loss: 39.3326, kld_loss: 24.8972
Train Epoch: 124 [0/110 (0%)]	Recons Loss: 99.435875; KLD Loss: 99.435875
Train Epoch: 124 [64/110 (50%)]	Recons Loss: 94.302933; KLD Loss: 94.302933
====> Epoch: 124 Average recons loss: 3.5488, kld loss: 12.6803
====> Epoch: 124 Eta is now 0.6065 due to sufficient recons results
====> Test set recons loss: 35.7266, kld_loss: 24.7925
Train Epoch: 125 [0/110 (0%)]	Recons Loss: 108.938972; KLD Loss: 1

...

====> Test set recons loss: 38.8850, kld_loss: 1.7529
Train Epoch: 390 [0/110 (0%)]	Recons Loss: 75.418762; KLD Loss: 75.418762
Train Epoch: 390 [64/110 (50%)]	Recons Loss: 67.706970; KLD Loss: 67.706970
====> Epoch: 390 Average recons loss: 2.6803, kld loss: 1.9559
====> Test set recons loss: 42.7978, kld_loss: 1.7432
Train Epoch: 391 [0/110 (0%)]	Recons Loss: 69.329361; KLD Loss: 69.329361
Train Epoch: 391 [64/110 (50%)]	Recons Loss: 67.679123; KLD Loss: 67.679123
====> Epoch: 391 Average recons loss: 2.7603, kld loss: 2.0615
====> Test set recons loss: 44.3822, kld_loss: 1.8872
Train Epoch: 392 [0/110 (0%)]	Recons Loss: 74.416000; KLD Loss: 74.416000
Train Epoch: 392 [64/110 (50%)]	Recons Loss: 77.244934; KLD Loss: 77.244934
====> Epoch: 392 Average recons loss: 2.7954, kld loss: 2.0197
====> Test set recons loss: 44.9315, kld_loss: 1.8992
Train Epoch: 393 [0/110 (0%)]	Recons Loss: 64.325241; KLD Loss: 64.325241
Train Epoch: 393 [64/110 (50%)]	Recons Loss: 81.166748; KLD Loss: 81.16

====> Test set recons loss: 41.7056, kld_loss: 1.8632
Train Epoch: 420 [0/110 (0%)]	Recons Loss: 72.104492; KLD Loss: 72.104492
Train Epoch: 420 [64/110 (50%)]	Recons Loss: 69.301941; KLD Loss: 69.301941
====> Epoch: 420 Average recons loss: 2.6680, kld loss: 1.9688
====> Test set recons loss: 41.8257, kld_loss: 1.9242
Train Epoch: 421 [0/110 (0%)]	Recons Loss: 75.328613; KLD Loss: 75.328613
Train Epoch: 421 [64/110 (50%)]	Recons Loss: 77.982094; KLD Loss: 77.982094
====> Epoch: 421 Average recons loss: 2.7547, kld loss: 1.9821
====> Test set recons loss: 40.8772, kld_loss: 1.9060
Train Epoch: 422 [0/110 (0%)]	Recons Loss: 79.190292; KLD Loss: 79.190292
Train Epoch: 422 [64/110 (50%)]	Recons Loss: 68.261559; KLD Loss: 68.261559
====> Epoch: 422 Average recons loss: 2.5304, kld loss: 2.1186
====> Test set recons loss: 44.7621, kld_loss: 1.9294
Train Epoch: 423 [0/110 (0%)]	Recons Loss: 60.396660; KLD Loss: 60.396660
Train Epoch: 423 [64/110 (50%)]	Recons Loss: 66.558617; KLD Loss: 66.55

====> Test set recons loss: 44.8156, kld_loss: 1.8758
Train Epoch: 450 [0/110 (0%)]	Recons Loss: 72.776825; KLD Loss: 72.776825
Train Epoch: 450 [64/110 (50%)]	Recons Loss: 61.695877; KLD Loss: 61.695877
====> Epoch: 450 Average recons loss: 2.4770, kld loss: 1.9676
====> Test set recons loss: 43.2628, kld_loss: 1.8581
Train Epoch: 451 [0/110 (0%)]	Recons Loss: 59.484104; KLD Loss: 59.484104
Train Epoch: 451 [64/110 (50%)]	Recons Loss: 71.890900; KLD Loss: 71.890900
====> Epoch: 451 Average recons loss: 2.2834, kld loss: 1.8800
====> Test set recons loss: 41.7503, kld_loss: 2.1835
Train Epoch: 452 [0/110 (0%)]	Recons Loss: 73.065155; KLD Loss: 73.065155
Train Epoch: 452 [64/110 (50%)]	Recons Loss: 78.325302; KLD Loss: 78.325302
====> Epoch: 452 Average recons loss: 2.4885, kld loss: 1.8016
====> Test set recons loss: 49.4168, kld_loss: 1.8215
Train Epoch: 453 [0/110 (0%)]	Recons Loss: 71.850754; KLD Loss: 71.850754
Train Epoch: 453 [64/110 (50%)]	Recons Loss: 65.716415; KLD Loss: 65.71

====> Test set recons loss: 42.9613, kld_loss: 1.9616
Train Epoch: 480 [0/110 (0%)]	Recons Loss: 63.312038; KLD Loss: 63.312038
Train Epoch: 480 [64/110 (50%)]	Recons Loss: 83.755463; KLD Loss: 83.755463
====> Epoch: 480 Average recons loss: 2.5285, kld loss: 1.9981
====> Test set recons loss: 43.5024, kld_loss: 1.9444
Train Epoch: 481 [0/110 (0%)]	Recons Loss: 54.575157; KLD Loss: 54.575157
Train Epoch: 481 [64/110 (50%)]	Recons Loss: 71.923286; KLD Loss: 71.923286
====> Epoch: 481 Average recons loss: 2.2253, kld loss: 1.9091
====> Test set recons loss: 43.4638, kld_loss: 2.2716
Train Epoch: 482 [0/110 (0%)]	Recons Loss: 72.037292; KLD Loss: 72.037292
Train Epoch: 482 [64/110 (50%)]	Recons Loss: 72.236664; KLD Loss: 72.236664
====> Epoch: 482 Average recons loss: 2.5876, kld loss: 1.7849
====> Test set recons loss: 45.5515, kld_loss: 1.8997
Train Epoch: 483 [0/110 (0%)]	Recons Loss: 63.090298; KLD Loss: 63.090298
Train Epoch: 483 [64/110 (50%)]	Recons Loss: 61.066380; KLD Loss: 61.06

KeyboardInterrupt: 

In [ ]:
1048576/128/128

In [ ]:
model.train()
train_loss = 0
for batch_idx,( data, _) in enumerate(train_loader):
    data = data.to(device)
    data = data.reshape([-1,3, 128*128])
    optimizer.zero_grad()
    recon_batch, mu, logvar = model(data)
    loss = loss_function(recon_batch, data, mu, logvar)
    loss.backward()
    train_loss += loss.item()
    optimizer.step()
    if batch_idx % 100 == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader),
            loss.item() / len(data)))